In [3]:
import sqlite3

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [18]:
def get_price_change(product_name : 'str'):
    """Queries database for specific product's info and returns history of it's price changes."""    
    con = sqlite3.connect("DB.db")
    cursor = con.cursor()
    query = f'SELECT * FROM prices WHERE product = \'{product_name}\' ORDER BY date_start'
    
    df = pd.read_sql(query, con)
    
    cursor.close()
    con.close()
    
    prices_per_date = (df.date_start.tolist(), df.price.tolist())
    return prices_per_date

In [21]:
get_price_change('A')

(['1970-01-01',
  '2010-01-21',
  '2010-08-17',
  '2010-11-02',
  '2012-08-27',
  '2012-12-06',
  '2013-01-11',
  '2013-05-02',
  '2013-05-08',
  '2013-11-01',
  '2014-08-30',
  '2015-12-29',
  '2016-05-04',
  '2016-08-14',
  '2016-09-28',
  '2017-04-01'],
 [297.79,
  164.68,
  558.58,
  127.08,
  352.13,
  0.35,
  100.61,
  120.5,
  260.19,
  1747.84,
  59.84,
  118.27,
  550.62,
  339.35,
  210.19,
  111.23])

In [22]:
def get_revenue_std(product_name : 'str'):
    """Queries database for specific product's revenue history and returns it's standard deviation."""       
    con = sqlite3.connect("DB.db")
    cursor = con.cursor()
    
    query = f'SELECT revenue FROM revenue WHERE product = \'{product_name}\''
    
    result = cursor.execute(query).fetchall()
    prod_std = round(np.std(result), 2)
    
    cursor.close()
    con.close()
    
    return prod_std

In [23]:
get_revenue_std('B')

33461.19

In [36]:
def lin_reg(product_name : 'str', start_date : 'str', end_date : 'str'):
    """Trains a linear regression model for amount of sold product vs day in selected dates' interval."""
    con = sqlite3.connect("DB.db")
    cursor = con.cursor()
    
    query = f"""SELECT product, date, amount FROM revenue 
    WHERE (product = \'{product_name}\') AND (date BETWEEN \'{start_date}\' AND \'{end_date}\') ORDER BY date"""
    
    df = pd.read_sql(query, con)
    
    x = df.index.values.reshape(-1, 1) # I use index assuming data is consistent and reliable,
    #we'd have to get a date range and assign day numbers for each row if it's not
    y = df.amount.values.reshape(-1, 1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
    
    lrm = LinearRegression()
    model = lrm.fit(x_train, y_train)
    y_pred = lrm.predict(x_test)
    
    k = lrm.coef_
    b = lrm.intercept_
    
    cursor.close()
    con.close()
    
    return k, b

In [37]:
lin_reg('Q', '2019-01-01', '2019-03-31')

(array([[-0.11719955]]), array([25.21387926]))